In [2]:
# Part a)
# Fixed-Charge Transportation Problem

import numpy as np
import gurobipy as gp
from gurobipy import GRB

"""
capacity = [5,8,7]
demand = [3,6,6,5]
varcost = [[4,5,2,7],[5,8,6,2],[8,9,4,3]]
fixcost = [[2,10,1,5],[7,2,7,8],[4,0,3,9]]
"""

"""
file = open('43.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]

fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(data[i+3+locations+1].split('    ')[j]) for j in range(customers) ] for i in range(locations)]
"""


file = open('36.txt','r')
data = file.readlines()

[locations, customers] = [int(x) for x in data[0].split(' ')]
capacity = [int(data[1].split(' ')[i]) for i in range(locations) ]
demand = [int(data[2].split(' ')[j]) for j in range(customers) ]
fixcost = [ [ int(data[i+3].split('   ')[j]) for j in range(customers) ] for i in range(locations)]
varcost = [ [ int(x) for x in data[i+3+locations+1].split('    ') if x!=''] for i in range(locations)]



model = gp.Model("Fixed-Charge Transportation problem")

m = len(demand) #number of customers
n = len(capacity) #number of sources 

# Create variables
transport = {} # transport quantity
decision = {} # binary variables representing decision to open route or not

for i in range(n):
    for j in range(m):
        transport[i,j] = model.addVar(obj=varcost[i][j])
        
for i in range(n):
    for j in range(m):
        decision[i,j] = model.addVar(vtype = GRB.BINARY, obj=fixcost[i][j])
        
# supply constraints
for i in range(n):
    model.addConstr(sum(transport[i,j] for j in range(m)) <= capacity[i]) 
    
# Demand constraints
for j in range(m):
    model.addConstr(sum(transport[i,j] for i in range(n)) >= demand[j])
    
# Binding constraints
for i in range(n):
    for j in range(m):
        model.addConstr(transport[i,j] <= decision[i,j] * min(capacity[i], demand[j]))

#for j in range(m): model.addConstr(decision[n-1,m] == 1)
    
model.optimize()

print("Objective: " + str(model.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x) for x in range(n)]
columns = ['Customer ' + str(x) for x in range(m)]

solution = pd.DataFrame(index=index, columns=columns)

for i in range(n):
    for j in range(m):
        solution.iloc[i,j] = transport[i,j].x

print(solution)

#model.dispose()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 255 rows, 450 columns and 900 nonzeros
Model fingerprint: 0xc39f2e20
Variable types: 225 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 16937.000000
Presolve time: 0.00s
Presolved: 255 rows, 450 columns, 900 nonzeros
Variable types: 225 continuous, 225 integer (225 binary)

Root relaxation: objective 8.737553e+03, 323 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8737.55336    0   17 16937.0000 8737.55336  48.4%     -    0s
H    0     0                    11436.000000 8737.55336  23.6%     -    0s
H    0     0                    11181.000000 8737.55336  21.9%     -    0s
H    0     0   

In [3]:
model.objBound

NameError: name 'model' is not defined

In [5]:
# Part b)
# Warehouse Location Problem

import numpy as np
import gurobipy as gp
from gurobipy import GRB

varcost = [[1,2,10,9,6,7,3],
           [2,9,0,7,3,6,10],
           [7,6,1,5,3,10,5],
           [6,5,10,2,6,3,6],
           [6,4,6,3,7,2,6]]
fixcost = [5, 7, 5, 6, 5]


m = len(varcost[0]) #number of customers
n = len(fixcost) #number of sources 



model = gp.Model("Warehouse Location Problem")

# Create variables
transport = {} # fraction of demand of customer j served by source i
decision = {} # binary variables representing decision to open route or not

for i in range(n):
    for j in range(m):
        transport[i,j] = model.addVar(obj=varcost[i][j])

for i in range(n):
    decision[i] = model.addVar(vtype = GRB.BINARY, obj=fixcost[i])

# NO supply constraints
    
# Demand constraints
for j in range(m):
    model.addConstr(sum(transport[i,j] for i in range(n)) == 1) 

bigM = 100;
# Binding constraints
for i in range(n):
    model.addConstr(sum(transport[i,j] for j in range(m)) <= decision[i]*bigM)

model.optimize()


print("Objective: " + str(model.objVal))

print("Solution:")
import pandas as pd
index = ['Source ' + str(x+1) for x in range(n)]
columns = ['Customer ' + str(x+1) for x in range(m)]

solution = pd.DataFrame(index=index, columns=columns+['y'])

for i in range(n):
    for j in range(m):
        solution.iloc[i,j] = transport[i,j].x
    solution.iloc[i,m] = decision[i].x


print(solution)

print('\nRunning time: ', model.Runtime)

model.dispose()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 40 columns and 75 nonzeros
Model fingerprint: 0x851d0da3
Variable types: 35 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 12 rows, 40 columns, 75 nonzeros
Variable types: 35 continuous, 5 integer (5 binary)

Root relaxation: objective 1.842857e+01, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.42857    0    5          -   18.42857      -     -    0s
H    0     0                      36.0000000   18.42857  48.8%     -    0s
H    0     0                      35.0000000   18.42857  47.3%     -    0s
     0     0   23.85714    0    4   35.00000   23.85714  31.8%     -    0s
H    0 

In [ ]:
# Part c)
# ADD heuristic

varcost = [[1,2,10,9,6,7,3],
           [2,9,0,7,3,6,10],
           [7,6,1,5,3,10,5],
           [6,5,10,2,6,3,6],
           [6,4,6,3,7,2,6]]
fixcost = [5, 7, 5, 6, 5]


m = len(varcost[0]) #number of customers
n = len(fixcost) #number of sources 

import numpy as np

from time import process_time 
start_time = process_time()      #computational time benchmarking

#Start solution
trans_cost = np.sum(varcost, axis =1)
start = np.argmin(trans_cost + fixcost)

print('Starting location to open is location ' + str(start+1))

# define function to calculate improvement in cost between current locations and potential location 
def improvement(current, potential):
    improvement = 0
    for i in range(m): # for each customer
        new_cost = varcost[potential][i]
        current_cost = min([varcost[x][i] for x in current])
        if new_cost < current_cost:
            improvement += current_cost - new_cost
    return improvement - fixcost[potential]

#improvement iteration
potential = set(range(n)) #set of potential locations
potential.remove(start) #remove starting location from potential locations set
opened = [] #set of potential locations
opened.append(start)

#rule: iterate until potential set is empty (all locations is checked): forbidden location is deleted 
#      while opened location is added to opened set 
while len(potential) > 0:
    max_saving = 0
    best_location = None
    forbidden = [] #list of forbidden locations
    
    #check all potential locations
    for location in potential:
        if improvement(opened, location) < 0:
            forbidden = forbidden + [location] 
        else:
            if improvement(opened, location) > max_saving:
                max_saving = improvement(opened, location)
                best_location = location
    
    for x in forbidden: potential.remove(x) #forbid locations with negative saving
    
    if max_saving == 0:
        break #end iteration if there is no location with positive saving
    else:
        opened.append(best_location) #open location with highest positive saving
        potential.remove(best_location)


stop_time = process_time()


print('\nFinal solution:')

totalFixcost = sum(fixcost[x] for x in opened)
totalTranscost = sum( [min( [varcost[x][i] for x in opened] ) for i in range(m)] )
print('Objective: ' + str(totalFixcost + totalTranscost))

print('Locations to open is location ' + ' '.join([str(x+1) for x in opened]))

print('Running time: ', stop_time - start_time)

#print(opened)

In [ ]:
print('Comment:\n')
print('The solutions obtained from two methods are slightly different. While in both solutions, the number of opened locations are 3 with two of which are 5 and 1. The other source to open is location 3 in TPP and location 2 in ADD heuristic. Therefore, the objective value in TPP (optimal value, cost = 30) is also slightly better than in ADD heuristic (cost = 31). \nHowever, regarding computational time, it is clearly that ADD heuristic (~ 0.0s) runs faster than TPP method (~0.03s).')